In [ ]:
%%time
from src.data.make_dataset import load_list, get_stock_data
from src.models.StockModel import StockModel
window_size = 20
start_date = '2017-01-01'
end_date = '2024-09-01'
feature_columns = ['Close', 'Open', 'Volume', 'Range', 'Gap', 'RSI', 'MACD']
target = "Open"

# Load symbols
nasdaq_symbols = load_list("NASDAQ")
sp500_symbols = load_list("SP500")

# Test tickers, sp500 symbols not also in nasdaq
test_tickers = [item for item in sp500_symbols if item not in nasdaq_symbols]
test_tickers = test_tickers[:25]

#tickers = ['^GSPC', '^IXIC', 'AAPL', 'MSFT', 'NVDA', 'AMZN', 'AVGO', 'META', 'GOOGL', 'GOOG', 'MMM', 'ADBE', 'BWA', 'GD', 'IT']
#tickers = ['^GSPC', '^IXIC']
train_tickers = ['^IXIC'] + nasdaq_symbols
train_tickers = train_tickers[:51]

# Download data
combined_data = get_stock_data(train_tickers, "1d", start_date, end_date)
combined_data.info()
# Test data
test_data = get_stock_data(test_tickers, "1d", start_date, end_date)

layer_config = [(1,128),(1,256),(1,512),(1,1024),(2,128),(2,256),(2,512),(2,768),(3,128),(3,256),(3,512)]
for i in layer_config:
    print(f"XXXXXXXXXXXXXXXX Running {i[0]} {i[1]} layers XXXXXXXXXXXXXXXXXXXX")
    # Create and train model
    stock_model = StockModel(window_size=window_size, feature_columns=feature_columns, target_name=target, export=True)
    
    stock_model.train(combined_data, patience=10, epochs=150, graph=False, layers=i[0], units_per_layer=i[1])
    metrics_dict, mean_metrics = stock_model.evaluate_many(test_data, graph=False)
    print(metrics_dict)
    print(mean_metrics)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 94076 entries, 2017-01-03 00:00:00-05:00 to 2024-08-30 00:00:00-04:00
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          94076 non-null  float64
 1   High          94076 non-null  float64
 2   Low           94076 non-null  float64
 3   Close         94076 non-null  float64
 4   Volume        94076 non-null  int64  
 5   Dividends     94076 non-null  float64
 6   Stock Splits  94076 non-null  float64
 7   Ticker        94076 non-null  object 
dtypes: float64(6), int64(1), object(1)
memory usage: 6.5+ MB
XXXXXXXXXXXXXXXX Running 1 128 layers XXXXXXXXXXXXXXXXXXXX
Initializing model:
 - Window size: 20
 - Features: ['Close', 'Open', 'Volume', 'Range', 'Gap', 'RSI', 'MACD']
 - Target: Open
--- Preparing ^IXIC data using 20 window---
Feature columns: ['Close', 'Open', 'Volume', 'Range', 'Gap', 'RSI', 'MACD', 'MACD_Hist']
Target column: Open
Feature min [ 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 128)                 │          70,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 70,273 (274.50 KB)

 Trainable params: 70,273 (274.50 KB)

 Non-trainable params: 0 (0.00 B)

--- 1/51 Training model for ^IXIC ---
Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0146 - mape: 140.4412 - val_loss: 0.0168 - val_mape: 12.8992
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0415 - mape: 293.7636 - val_loss: 0.0170 - val_mape: 14.8616
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0075 - mape: 115.8603 - val_loss: 0.0014 - val_mape: 3.6506
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0016 - mape: 38.1701 - val_loss: 2.4561e-04 - val_mape: 1.3168
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0013 - mape: 35.4128 - val_loss: 1.9498e-04 - val_mape: 1.1947
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0010 - mape: 27.1385 - val_loss: 3.1825e-04 - val_mape: 1.5707
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0013 - mape: 36.6958 - val_loss: 2.1737e-04 - val_mape: 1.2861
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0011 - mape: 30.6412 - val_loss: 2.8406

In [ ]:
test_tickers = [item for item in sp500_symbols if item not in nasdaq_symbols]
test_tickers = test_tickers[:5]

#tickers = ['^GSPC', '^IXIC', 'AAPL', 'MSFT', 'NVDA', 'AMZN', 'AVGO', 'META', 'GOOGL', 'GOOG', 'MMM', 'ADBE', 'BWA', 'GD', 'IT']
#tickers = ['^GSPC', '^IXIC', '^DJI']
# Test data
test_data = get_stock_data(["^GSPC", "^DJI"], "1d", start_date, end_date)

metrics_dict, mean_metrics = stock_model.evaluate_many(test_data, graph=True)
print(metrics_dict)
print(mean_metrics)


In [4]:
import time
inicio = time.time()
for i in range(100000000):
    pass
fin = time.time()
tiempo = fin-inicio
# Convertimos el tiempo a horas, minutos y segundos
horas = int(tiempo // 3600)
minutos = int((tiempo % 3600) // 60)
segundos = int(tiempo % 60)

# Imprimimos el tiempo en formato hh:mm:ss
print(f"Tiempo transcurrido: {horas:02d}:{minutos:02d}:{segundos:02d}")

Tiempo transcurrido: 00:00:02
